# 01 OS GeoDataViz Colour Palettes
---

Using the OS GeoDataViz Toolkit to assist with the styling of location data insights.

### OS GeoDataViz Toolkit
---

The [OS GeoDataViz (GDV) Toolkit](https://github.com/OrdnanceSurvey/GeoDataViz-Toolkit) provides a set of resources to help you communicate your data effectively through the design of compelling visuals.

The toolkit provides a series of colour palettes with provide a great starting point for thematic map design.
The palettes can be programatically accessed via a JSON copy in the repo.

### Qualitative Styling
---

When styling qualitative (categorical) data we need to be mindful of the number of distinct classes. The GDV qualitative colours supports up to eight classes. Beyond approx. ten classes it becomes very difficult to design a palette that preserves the seperation between colours allowing clear class identification.

In [ ]:
import geopandas as gpd
import requests
from datetime import datetime
from folium import Map
from folium.plugins import FloatImage
from matplotlib.colors import ListedColormap

###  Create GeoDataFrame from GeoPackage (GPKG)

In [ ]:
# Create a GeoPandas GeoDataFrame from a GeoPackage (GPKG)
osogs = gpd.read_file(
    filename="../../data/ordnance-survey/os-open-greenspace-gb.gpkg",
    # GPKG layer
    layer="greenspace_site",
)

###  Spatially subset GeoDataFrame

Using coordinate-based indexer to spatially subset by bounding box (BBOX).

In [ ]:
# Coordinate-based indexer to select by intersection with BBOX
# Greater London BBOX
osogs_filtered = osogs.cx[
    503568.1996:561957.4962, 155850.7975:200933.9026
]  # xmin:xmax, ymin:ymax

In [ ]:
osogs_filtered.head()

In [ ]:
osogs_filtered["function"].unique()

In [ ]:
# Count number of unique function values
osogs_filtered["function"].nunique()

### Subset GeoDataFrame by function

Reduce the number of function categories to support qualitative styling colour palette integration.

In [ ]:
# Subset GeoDataFrame by function excluding 'Play Space' and 'Tennis Court'
osogs_filtered = osogs_filtered.loc[
    # ~ anti conditional
    ~(osogs_filtered["function"].isin(["Play Space", "Tennis Court"]))
]

In [ ]:
# List unique function classes
osogs_filtered["function"].unique()

In [ ]:
# Count number of unique function values
osogs_filtered["function"].nunique()

### OS Maps API ZXY resource

In [ ]:
# OS Maps API layer name
# Example uses Light Style in Web Mercator (EPSG:3857) projection
layer = "Light_3857"
# OS Data Hub project API key
key = "frKhvBUiMB5DGwl3pGb2GzcOz6ApgyP0"
# OS Data Hub base path - https://api.os.uk
# OS Maps API ZXY end point path - /maps/raster/v1/zxy/
url = f"https://api.os.uk/maps/raster/v1/zxy/{layer}/{{z}}/{{x}}/{{y}}.png?key={key}"

# OS logo image
logo_url = "https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.svg"

# Folium FloatImage plugin for displaying image on Map
image = FloatImage(logo_url, bottom=1, left=1)

### Create custom folium map

In [ ]:
# Create Folium map
m = Map(
    location=[
        51.507,
        -0.105,
    ],  # Map centre coordinates (by convention latitude (y), longitude (x))
    tiles=url,
    attr=f"Contains OS data &copy; Crown copyright and database rights {datetime.now().year}",  # OS Data Hub attribution statement
    min_zoom=7,  # See EPSG:3857 Tile Matrix Set - https://osdatahub.os.uk/docs/wmts/technicalSpecification
    max_zoom=16,
    zoom_start=12,
)

# Add image to Map
image.add_to(m)

### OS GeoDataViz colour palettes

The OS [GeoDataViz toolkik](https://github.com/OrdnanceSurvey/GeoDataViz-Toolkit/tree/master/Colours) provides qualitative, sequential, and diverging colour palettes to support GDV applications.

In [ ]:
# GDV colour palettes JSON file
gdv = "https://raw.githubusercontent.com/OrdnanceSurvey/GeoDataViz-Toolkit/master/Colours/GDV-colour-palettes.json"

requests.get(gdv, verify=False).json()

In [ ]:
# Make HTTP GET request, decode JSON, and access by JSON query
gdv_qual = requests.get(gdv, verify=False).json()["qualitative"]["lookup"]

# Generate list of integers (1-8) as strings
colours = [str(x) for x in range(1, 9)]

# Create Matplotlib colour map from GDV colours
gdv_qual_cmap = ListedColormap(
    colors=[gdv_qual[c] for c in colours], name="gdv-qual-cmap"
)

# Return colour map
gdv_qual_cmap

In [ ]:
colours

### Plot GeoDataFrame ontop of folium map

Employ categorical styling via OS GeoDataViz (GDV) qualitative colour palette.

In [ ]:
# Add GeoDataFrame to folium map
osogs_filtered.explore(
    column="function",  # Qualitative styling based on 'function' column values
    m=m,  # Custom folium map
    cmap=gdv_qual_cmap,# OS GDV colour map
    popup = False
)  